# Immigration Data Lake
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
# Read in the data here
fname = 'data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [ ]:
df.head()

In [ ]:
# save to csv in data folder
df.to_csv('data/i94_apr16_sub.csv')

In [ ]:
# save a sample of the csv to the data folder
df.sample(1000).to_csv('data/i94_apr16_sub_sample.csv')

In [ ]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
#write to parquet
df_spark.write.parquet("data/sas_data")
df_spark=spark.read.parquet("data/sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Immigration Data

In [ ]:
# read immigration data
imm_df = spark.read.format('com.github.saurfang.sas.spark').load('data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [ ]:
# immigration data schema
imm_df.printSchema()

In [ ]:
# | I94RES | 3 digit code for country of residence |
imm_df.groupBy('I94RES').count().show()

In [ ]:
# | I94ADDR | state of arrival |
imm_df.groupBy('I94ADDR').count().show()

In [ ]:
# | I94VISA | visa codes collapsed into three categories: 1 = Business; 2 = Pleasure; 3 = Student |
imm_df.groupBy('I94VISA').count().show()

In [ ]:
imm_df.groupBy('I94PORT').count().show()

In [ ]:
imm_df.select('I94VISA').show()

In [ ]:
imm_df.groupBy('I94YR').count().show()

In [ ]:
gr = imm_df.groupBy("INSNUM").agg(countDistinct("GENDER"))
gr.show()

In [ ]:
gr.groupBy('count(GENDER)').count().show()

In [ ]:
# Wrong date times, because its a SAS numeric time field
from pyspark.sql.functions import col , column
imm_df.withColumn("ts", col("ARRDATE").cast("timestamp")).select('ts').show()

#### Weather Data

In [ ]:
weather_df = spark.read.options(header='true', inferSchema='true').csv('data/GlobalLandTemperaturesByState.csv')

In [ ]:
weather_df.printSchema()

In [ ]:
weather_df.show(10)

In [ ]:
weather_df.groupBy('State').count().show()

#### Demographics data

In [ ]:
# demo
demo_df = spark.read.options(header='true', inferSchema='true', delimiter=';').csv('data/us-cities-demographics.csv')

In [ ]:
demo_df.limit(10).toPandas()

#### Airport data

In [ ]:
# airport
airport_df = spark.read.options(header='true', inferSchema='true').csv('data/airport-codes_csv.csv')

In [ ]:
airport_df.show(5)

In [ ]:
airport_df.groupBy('iso_country').count().show()

In [ ]:
airport_df.filter((airport_df["ident"] == "") | airport_df["ident"].isNull()).count()

In [ ]:
airport_df.groupBy('ident').count().count()

In [ ]:
airport_df.count()

#### Cleaning Steps
Document steps necessary to clean the data

#### Immigration data cleaning

In [ ]:
imm_df.printSchema()

In [ ]:
# rename cicid to id
# rename i94yr to year
# rename i94mon to month
# read mapping for code to country of origin, and map the I94CIT var to the actual country of origin (do later)

In [ ]:
imm_fact = imm_df.select(col("cicid").alias("id"),
                          col("i94yr").alias("year"),
                          col("i94mon").alias("month"),
                          col("I94CIT").alias("country_of_origin"),
                          col("I94ADDR").alias("state_of_arrival"))

In [ ]:
imm_fact.show()

In [ ]:
person_dim = imm_df.select(col("cicid").alias("id"),
                          col("GENDER").alias("gender"),
                          col("I94BIR").alias("I94BIR"))

In [ ]:
person_dim.show()

In [ ]:
demo_df.show(5)

In [ ]:
demo_df = demo_df.withColumn("id", monotonically_increasing_id())

In [ ]:
demo_df.printSchema()

In [ ]:
demographics_dim = demo_df.select(col("id"),
                                  col("State").alias("state"),
                                  col("MEDIAN AGE").alias("median_age"),
                                  col("Total Population").alias("total_population"))

In [ ]:
demographics_dim.show(5)

In [ ]:
airport_df.printSchema()

In [ ]:
airport_df.limit(10).toPandas()

In [ ]:
airports_dim = airport_df.select(col("ident").alias("id"),
                                  col("iso_country").alias("country"),
                                  col("MEDIAN AGE").alias("median_age"),
                                  col("Total Population").alias("total_population"))

### Create country code mapping table using the I94 description file

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/i94_country_code_mapping.csv", header=None)

In [ ]:
df = df.replace(np.nan, '', regex=True)
cols = [1,2,3,4,5,6,7,8,9,10]
df['country'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df = df[[0, 'country']]
df.columns = ['code', 'country']
df = df[df['code'] != '']
df['code'] = df['code'].astype(int)
df.to_csv('data/code_to_country_mapping.csv', index=False)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [1]:
# Imports
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql import functions as F
from pyspark.sql.functions import col, create_map, lit, expr, to_date
from itertools import chain

In [2]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12")\
.enableHiveSupport().getOrCreate()

In [ ]:
# Read all tables
immigration_staging = spark.read.options(header='true', inferSchema='true').csv('data/i94_apr16_sub_sample.csv')
country_code_mapping_staging = spark.read.options(header='true', inferSchema='true').csv('data/code_to_country_mapping.csv')
demographics_staging = spark.read.options(header='true', inferSchema='true', delimiter=';').csv('data/us-cities-demographics.csv')
weather_staging = spark.read.options(header='true', inferSchema='true').csv('data/GlobalLandTemperaturesByState.csv')
airport_staging = spark.read.options(header='true', inferSchema='true').csv('data/airport-codes_csv.csv')

In [ ]:
transportation_mapping = {1: 'air', 2: 'sea', 3: 'land', 4: 'not reported'}
transportation_mapping_expr = create_map([lit(x) for x in chain(*transportation_mapping.items())])

state_mapping = {'AL':'ALABAMA',
                    'AK':'ALASKA',
                    'AZ':'ARIZONA',
                    'AR':'ARKANSAS',
                    'CA':'CALIFORNIA',
                    'CO':'COLORADO',
                    'CT':'CONNECTICUT',
                    'DE':'DELAWARE',
                    'DC':'DIST. OF COLUMBIA',
                    'FL':'FLORIDA',
                    'GA':'GEORGIA',
                    'GU':'GUAM',
                    'HI':'HAWAII',
                    'ID':'IDAHO',
                    'IL':'ILLINOIS',
                    'IN':'INDIANA',
                    'IA':'IOWA',
                    'KS':'KANSAS',
                    'KY':'KENTUCKY',
                    'LA':'LOUISIANA',
                    'ME':'MAINE',
                    'MD':'MARYLAND',
                    'MA':'MASSACHUSETTS',
                    'MI':'MICHIGAN',
                    'MN':'MINNESOTA',
                    'MS':'MISSISSIPPI',
                    'MO':'MISSOURI',
                    'MT':'MONTANA',
                    'NC':'N. CAROLINA',
                    'ND':'N. DAKOTA',
                    'NE':'NEBRASKA',
                    'NV':'NEVADA',
                    'NH':'NEW HAMPSHIRE',
                    'NJ':'NEW JERSEY',
                    'NM':'NEW MEXICO',
                    'NY':'NEW YORK',
                    'OH':'OHIO',
                    'OK':'OKLAHOMA',
                    'OR':'OREGON',
                    'PA':'PENNSYLVANIA',
                    'PR':'PUERTO RICO',
                    'RI':'RHODE ISLAND',
                    'SC':'S. CAROLINA',
                    'SD':'S. DAKOTA',
                    'TN':'TENNESSEE',
                    'TX':'TEXAS',
                    'UT':'UTAH',
                    'VT':'VERMONT',
                    'VI':'VIRGIN ISLANDS',
                    'VA':'VIRGINIA',
                    'WV':'W. VIRGINIA',
                    'WA':'WASHINGTON',
                    'WI':'WISCONSON',
                    'WY':'WYOMING' ,
                    '99':'All Other Codes'}
state_mapping = dict((k, v.title()) for k, v in state_mapping.items())
state_mapping_expr = create_map([lit(x) for x in chain(*state_mapping.items())])

visa_mapping = {1: 'business', 2: 'pleasure', 3: 'student'}
visa_mapping_expr = create_map([lit(x) for x in chain(*visa_mapping.items())])

immigration_fact =  immigration_staging.select(col("cicid").alias("id").cast("int"),
                    col("I94YR").alias("year").cast("int"),
                    col("I94MON").alias("month").cast("int"),
                    col("I94PORT").alias("port_of_entry"),
                    col("ARRDATE").alias("arrival_date").cast("int"),
                    col("I94MODE").alias("mode_of_transportation").cast("int"),
                    col("I94ADDR").alias("state_of_arrival_code"),
                    col("DEPDATE").alias("departure_date").cast("int"),
                    col("I94VISA").alias("visa_type").cast("int"),) \
.withColumn("mode_of_transportation", transportation_mapping_expr[col("mode_of_transportation")]) \
.withColumn("state_of_arrival", state_mapping_expr[col("state_of_arrival_code")]) \
.withColumn("sas_date", to_date(lit("01/01/1960"), "MM/dd/yyyy")) \
.withColumn("arrival_date", expr("date_add(sas_date, arrival_date)")) \
.withColumn("departure_date", expr("date_add(sas_date, departure_date)")) \
.withColumn("visa_type", visa_mapping_expr[col("visa_type")]) \
.drop('sas_date')

In [ ]:
immigration_fact.show(5)

In [ ]:
person_dim = immigration_staging.join(country_code_mapping_staging, 
                         immigration_staging["I94CIT"] == country_code_mapping_staging["code"], 
                         "left").withColumnRenamed("country", "country_of_origin").drop('code')
person_dim = person_dim.join(country_code_mapping_staging, 
                         person_dim["I94RES"] == country_code_mapping_staging["code"], 
                         "left").withColumnRenamed("country", "country_of_residence").drop('code')
person_dim = person_dim.select(col("cicid").alias("id").cast("int"),
                                            col("GENDER").alias("gender"),
                                            col("I94BIR").alias("age").cast("int"),
                                            col("country_of_origin").alias("country_of_origin"),
                                            col("country_of_residence").alias("country_of_residence"),
                                            col("BIRYEAR").alias("year_of_birth").cast("int"))

In [ ]:
person_dim.show(5)

In [ ]:
# Demographics dimension
demographics_dim = demographics_staging.withColumn("id", monotonically_increasing_id()) \
                    .select(col("id"),
                            col("State").alias("state"),
                            col("Median Age").alias("median_age"),
                            col("Female Population").alias("female_population"),
                            col("Number of Veterans").alias("number_of_veterans"),
                            col("Foreign-born").alias("foreign_born"),
                            col("Average Household Size").alias("average_household_size"),
                            col("State Code").alias("state_code"),
                            col("Race").alias("race"),
                            col("Count").alias("count"))


In [ ]:
demographics_dim.show(5)

In [ ]:
demographics_dim.printSchema()

In [ ]:
weather_dim = weather_staging.withColumn("id", monotonically_increasing_id()) \
                              .select(col("id"),
                                     col("dt").alias("date"),
                                     col("AverageTemperature").alias("average_temperature"),
                                     col("AverageTemperatureUncertainty").alias("average_temperature_uncertainty"),
                                     col("State").alias("state"),
                                     col("Country").alias("country"))

In [ ]:
weather_dim.show(5)

In [ ]:
airport_dim = airport_staging

In [ ]:
airport_staging.printSchema()

In [ ]:
# airport_staging.show(5)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.